<a href="https://colab.research.google.com/github/ZilingLin/HARP/blob/master/HARP_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#using the mathod of fake-weight to improve ANRL

##import google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/graduate_work
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/graduate_work
config.py  evaluation.py  main.py   node2vec.py   README.md         utils.py
embed/     graph/         model.py  __pycache__/  requirements.txt


## import tensorflow and other bags

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import numpy as np
import networkx as nx
import random
import math
import time
import os
import warnings
import collections
import scipy.io as scio
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import *
from sklearn.metrics.pairwise import cosine_similarity

TensorFlow 1.x selected.
1.15.2


###needed in harp

In [0]:
from collections import deque

##Flags and tf envriomment

In [0]:
tf.app.flags.DEFINE_string('datasets', 'citeseer', 'datasets descriptions')
tf.app.flags.DEFINE_string('inputEdgeFile', 'graph/citeseer.edgelist', 'input graph edge file')
tf.app.flags.DEFINE_string('inputFeatureFile', 'graph/citeseer.feature', 'input graph feature file')
tf.app.flags.DEFINE_string('inputLabelFile', 'graph/citeseer.label', 'input graph label file')
tf.app.flags.DEFINE_string('outputEmbedFile', 'embed/citeseer.embed', 'output embedding result')
tf.app.flags.DEFINE_integer('dimensions', 128, 'embedding dimensions')
tf.app.flags.DEFINE_integer('feaDims', 3703, 'feature dimensions')
tf.app.flags.DEFINE_integer('walk_length', 80, 'walk length')
tf.app.flags.DEFINE_integer('num_walks', 10, 'number of walks')
tf.app.flags.DEFINE_integer('window_size', 10, 'window size')
tf.app.flags.DEFINE_float('p', 1.0, 'p value')
tf.app.flags.DEFINE_float('q', 1.0, 'q value')
tf.app.flags.DEFINE_boolean('weighted', False, 'weighted edges')
tf.app.flags.DEFINE_boolean('directed', False, 'undirected edges')

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
config_tf = tf.ConfigProto()
config_tf.gpu_options.allow_growth = True

add something that i dont know what it do but required by colab

In [0]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

##configs

In [0]:
class Config(object):
    def __init__(self):
        # hyperparameter
        self.struct = [None, 1000, 500, None]
        self.alpha = 10
        self.reg = 4

        # parameters for training
        self.batch_size = 512
        self.num_sampled = 10
        self.max_iters_finaltrain = 20000
        self.max_iters_pretrain = 8000
        self.learning_rate = 1e-4


##utils

In [0]:

def read_graph(FLAGS, edgeFile):
    print('loading graph...')

    if FLAGS.weighted:
        G = nx.read_edgelist(edgeFile, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
    else:
        G = nx.read_edgelist(edgeFile, nodetype=int, create_using=nx.DiGraph())
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight'] = 1
            # G.add_edge(edge[1], edge[0])
            # G[edge[1]][edge[0]]['weight'] = 1

    if not FLAGS.directed:
        G = G.to_undirected()

    return G

def read_feature_mat(inputFileName):
    data = scio.loadmat(inputFileName)
    return data['feature']



def read_feature(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()

    features = []
    for line in lines[1:]:
        l = line.strip('\n\r').split(' ')
        features.append(l)
    features = np.array(features, dtype=np.float32)

    return features


def write_embedding(embedding_result, outputFileName):
    f = open(outputFileName, 'w')
    N, dims = embedding_result.shape

    for i in range(N):
        s = ''
        for j in range(dims):
            if j == 0:
                s = str(i) + ',' + str(embedding_result[i, j])
            else:
                s = s + ',' + str(embedding_result[i, j])
        f.writelines(s + '\n')
    f.close()


def write_pairs(pairs, outputFileName):
    f = open(outputFileName, 'w')
    length = len(pairs)

    # test
    for i in range(length):
        s = str(pairs[i, 0]) + ' ' + str(pairs[i, 1])
        f.writelines(s + '\n')
    f.close()


def read_pairs(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()

    pairlist = []
    for line in lines:
        l = line.strip('\n\r').split(' ')
        pair = [int(l[0]), int(l[1])]
        pairlist.append(pair)
    return pairlist


def write_weights(nx_G, outputfile):
    f = open(outputfile, 'w')
    nodes = nx_G.nodes()
    for node in nodes:
      neighbors = list(nx_G.neighbors(node))
      neighbors_length = len(neighbors)
      if neighbors_length != 0:
        for n in neighbors:
          s = str(node) + ' ' + str(n) + ' ' + str(nx_G[node][n]['weight'])
          f.writelines(s + '\n')
    f.close()

##evaluation

In [0]:

def read_label(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()
    N = len(lines)
    y = np.zeros(N, dtype=int)
    for line in lines:
        l = line.strip('\n\r').split(' ')
        y[int(l[0])] = int(l[1])

    return y


def multiclass_node_classification_eval(X, y, ratio=0.2, rnd=2018):
    warnings.filterwarnings('ignore')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=rnd)
    clf = LinearSVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    macro_f1 = f1_score(y_test, y_pred, average='macro')
    micro_f1 = f1_score(y_test, y_pred, average='micro')

    return macro_f1, micro_f1


def link_prediction_ROC(inputFileName, Embeddings):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()

    X_test = []

    for line in lines:
        l = line.strip('\n\r').split(' ')
        X_test.append([int(l[0]), int(l[1]), int(l[2])])

    y_true = [X_test[i][2] for i in range(len(X_test))]
    y_predict = [
        cosine_similarity(Embeddings[X_test[i][0], :].reshape(1, -1), Embeddings[X_test[i][1], :].reshape(1, -1))[0, 0]
        for i in range(len(X_test))]
    auc = roc_auc_score(y_true, y_predict)

    if auc < 0.5:
        auc = 1 - auc

    return auc


def node_classification_F1(Embeddings, y, ratio):
    macro_f1_avg = 0
    micro_f1_avg = 0
    for i in range(10):
        rnd = np.random.randint(2018)
        macro_f1, micro_f1 = multiclass_node_classification_eval(Embeddings, y, ratio, rnd)
        macro_f1_avg += macro_f1
        micro_f1_avg += micro_f1
    macro_f1_avg /= 10
    micro_f1_avg /= 10
    print('Macro_f1 average value: ' + str(macro_f1_avg))
    print('Micro_f1 average value: ' + str(micro_f1_avg))


##node2vec

In [0]:
class Graph():
    def __init__(self, nx_G, is_directed, p, q):
        self.G = nx_G
        self.is_directed = is_directed
        self.p = p
        self.q = q

    def node2vec_walk(self, walk_length, start_node):
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(
                        cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0],
                                               alias_edges[(prev, cur)][1])]
                    walk.append(next)
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print('simulating random walk...')
        for walk_iter in range(num_walks):
            # print(str(walk_iter + 1), '/', str(num_walks))
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(
                    walk_length=walk_length, start_node=node))
        return walks

    def get_alias_edge(self, src, dst):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G
        p = self.p
        q = self.q

        # src是上一结点，dst是当前结点，dst_nbr是下一可能的结点
        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            if dst_nbr == src:
                unnormalized_probs.append(G[dst][dst_nbr]['weight'] / p)
            elif G.has_edge(dst_nbr, src):
                unnormalized_probs.append(G[dst][dst_nbr]['weight'])
            else:
                unnormalized_probs.append(G[dst][dst_nbr]['weight'] / q)
        norm_const = sum(unnormalized_probs)
        normalized_probs = [
            float(u_prob) / norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        is_directed = self.is_directed

        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr]['weight']
                                  for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs = [
                float(u_prob) / norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)

        alias_edges = {}
        if is_directed:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
        else:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
                alias_edges[(edge[1], edge[0])] = self.get_alias_edge(
                    edge[1], edge[0])

        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges


def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q


def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]


##harp

###bfs
broad first search

In [0]:
def bfs(G, cur_node):
    component = []
    q = deque()
    q.append(cur_node)
    G.nodes[cur_node]['visit'] = True

    while len(q) > 0:
        head = q.popleft()
        component.append(head)
        for nbr in list(G.neighbors(head)):
            if G.nodes[nbr]['visit'] == False:
                G.nodes[nbr]['visit'] = True
                q.append(nbr)
    return component

###check if it is connected

In [0]:
def is_connected(G):
    nodes = list(G.nodes())
    for node in nodes:
        G.nodes[node]['visit'] = False
    component = bfs(G, nodes[0])
    if len(component) == len(nodes):
        return True
    else:
        return False

###Group Collapsing
Every member of the group is permitted to bring their friends to the group, but each of the friends should pass the exam of the group which is the enough vote from the member of the group.

In [0]:
# def group_collapsing(G):
#     groups = []
#     nodes = list(G.nodes())
#     for node in nodes:
#         G.nodes[node]['visit'] = False

#     for node in nodes:
#         if G.nodes[node]['visit'] == False:
#             group = [node]
#             candidates = set(G.neighbors(node))
#             G.nodes[node]['visit'] = True
#             while len(candidates) > 0:
#                 candidate = candidates.pop()
#                 if G.nodes[candidate]['visit'] == False:
#                     vote = 1
#                     for member in group:
#                         if candidate in G.neighbors(member):
#                             vote += 1
#                     if vote > len(group) / 2:
#                         group.append(candidate)
#                         candidates = candidates.union(set(G.neighbors(candidate)))
#                         candidates = candidates.difference(set(group))
#                         G.nodes[candidate]['visit'] = True
#         groups.append(group)

#     return groups

###edge collapsing

In [0]:
def edge_collapsing(G):
    groups = []
    nodes = np.random.permutation(list(G.nodes()))
    for node in nodes:
        G.nodes[node]['visit'] = False
    
    for node in nodes:
        if G.nodes[node]['visit'] == False:
            group = []
            for nbr in np.random.permutation(list(G.neighbors(node))):
                if G.nodes[nbr]['visit'] == False:
                    G.nodes[nbr]['visit'] = True
                    group.append(nbr)
                    break
            G.nodes[node]['visit'] = True
            group.append(node)
            groups.append(group)

    return groups

###star collapsing

In [0]:
def star_collapsing(G):
    groups = []
    nodes = np.random.permutation(list(G.nodes()))
    for node in nodes:
        G.nodes[node]['visit'] = False
    
    for node in nodes:
        neighbors = np.random.permutation(list(G.neighbors(node)))
        group = []
        for neighbor in neighbors:
            if G.nodes[neighbor]['visit'] == False:
                group.append(neighbor)
                G.nodes[neighbor]['visit'] = True
                if len(group) == 2:
                    groups.append(group)
                    group = []
        if len(group) == 1:
            groups.append(group)
            group = []

    return groups

###build recursive graph

In [0]:
def build_regraph(G, groups):
    regraph = nx.Graph()
    # print(groups)
    for i, group in enumerate(groups):
        group.sort()
        # print(i, len(groups))
        for member in group:
            G.nodes[member]['collapsing'] = i
    
    edges = list(G.edges())
    for edge in edges:
        if G.nodes[edge[0]]['collapsing'] != G.nodes[edge[1]]['collapsing']:
            # print(G.nodes[edge[0]]['collapsing'], G.nodes[edge[1]]['collapsing'])
            regraph.add_edge(G.nodes[edge[0]]['collapsing'], G.nodes[edge[1]]['collapsing'])
    # print('len of the groups', len(groups))
    # print('origin:', is_connected(G))
    # print('after build regraph', is_connected(regraph))
    
    # if graph is unweighted
    edges = list(regraph.edges())
    for edge in edges:
        regraph[edge[0]][edge[1]]['weight'] = 1
    
    nodes = list(regraph.nodes())
    nodes.sort(reverse=True)
    for node in nodes:
        regraph.nodes[node]['visit'] = False

    # print('nodes', nodes)
    # print('len of nodes', len(regraph.nodes()))
    # print('next')

    return regraph

###prolongate

In [0]:
def prolongate(G, embedding_input, dimension):
    embedding = np.zeros([G.number_of_nodes(), dimension], dtype = np.float32)
    # print(np.shape(embedding_input), np.shape(embedding), dimension)
    nodes = list(G.nodes())
    for node in nodes:
        # print(node, G.nodes[node]['collapsing'])
        embedding[node] = embedding_input[G.nodes[node]['collapsing']]
    return embedding

###only skipgram model

In [0]:
class Skipgram_Model:
    def __init__(self, config, N, dims, embedding_input = None):
        tf.reset_default_graph() 
        self.config = config
        self.N = N
        self.dims = dims

        self.labels = tf.placeholder(tf.int32, shape=[None, 1])
        self.inputs = tf.placeholder(tf.int32, shape=[None])

        # Create the embedding variable (each row represent a word embedding vector)
        if embedding_input is None:
            self.embedding = tf.get_variable('embedding', [self.N, self.dims], initializer=tf.contrib.layers.xavier_initializer())
        else:
            self.embedding = tf.Variable(embedding_input)

        # Lookup the corresponding embedding vectors for each sample in X
        self.Y = tf.nn.embedding_lookup(self.embedding, self.inputs)

        # for test
        # self.test = tf.reduce_sum(tf.add_n([self.Y]))

        ############ define variables for skipgram  ####################
        # construct variables for nce loss
        self.nce_weights = tf.get_variable('nce_weights', [self.N, self.dims],
                                           initializer=tf.contrib.layers.xavier_initializer())
        self.nce_biases = tf.get_variable('nce_biases', [self.N], initializer=tf.zeros_initializer())

        self.loss_sg = self.make_skipgram_loss()

        # compute gradients for skipgram
        self.train_opt_sg = tf.train.AdamOptimizer(self.config.learning_rate).minimize(self.loss_sg)

    def make_skipgram_loss(self):
        loss = tf.reduce_sum(tf.nn.sampled_softmax_loss(
            weights=self.nce_weights,
            biases=self.nce_biases,
            labels=self.labels,
            inputs=self.Y,
            num_sampled=self.config.num_sampled,
            num_classes=self.N))
        return loss

####train

In [0]:
def train(G, FLAGS, max_iters, embedding_input = None):
    print ('training graph with %d nodes and %d edges' % 
           (G.number_of_nodes(), G.number_of_edges()))
    node2vec_G = Graph(G, FLAGS.directed, FLAGS.p, FLAGS.q)
    node2vec_G.preprocess_transition_probs()
    walks = node2vec_G.simulate_walks(FLAGS.num_walks, FLAGS.walk_length)

    nodes = list(G.nodes())
    N = len(nodes)
    dims = FLAGS.dimensions
    config = Config()
    my_context_iter = my_graph_context_batch_iter(walks, FLAGS, config.batch_size)
    model = Skipgram_Model(config, N, dims, embedding_input = embedding_input)

    init = tf.global_variables_initializer()
    # sess = tf.Session(config=config_tf)
    # sess.run(init)
    with tf.Session(config=config_tf) as sess:

        # Run the initializer
        sess.run(init)

        batch_size = config.batch_size
        idx = 0
        print_every_k_iterations = 1000
        start = time.time()
        loss_sg = 0

        for iter_cnt in range(max_iters):
            idx += 1

            # train for skip-gram model
            batch_index, batch_labels = next(my_context_iter)
            feed_dict = {model.inputs: batch_index, model.labels: batch_labels}
            _, loss_sg_value = sess.run([model.train_opt_sg, model.loss_sg], feed_dict=feed_dict)
            loss_sg += loss_sg_value

            if idx % print_every_k_iterations == 0:
                end = time.time()
                print('iterations: %d' % idx + ', time elapsed: %.2f, ' % (end - start), end='')
                print('loss: %.2f, ' % (loss_sg / idx))

        print('optimization finished...')
        embedding_result = sess.run(model.embedding)
    #     print(np.shape(embedding_result))
    # print(np.shape(embedding_result))
    return embedding_result

###graph_coarsening

In [0]:
def graph_coarsening(original_graph):
    recursive_graphs = []
    recursive_graphs.append(original_graph)
    while recursive_graphs[-1].number_of_nodes() > 100:
        if len(recursive_graphs) % 2 == 1:
            groups = star_collapsing(recursive_graphs[-1])
            graph = build_regraph(recursive_graphs[-1], groups)
        else:
            groups = edge_collapsing(recursive_graphs[-1])
            graph = build_regraph(recursive_graphs[-1], groups)
        recursive_graphs.append(graph)
        print('len of recursive_graphs and size', len(recursive_graphs), recursive_graphs[-1].number_of_nodes())
    return recursive_graphs

###build_subgraphs

In [0]:
def build_subgraphs(G):
    subgraphs = []
    
    nodes = list(G.nodes())
    for node in nodes:
        G.nodes[node]['visit'] = False

    components = []
    smallone = []
    for node in nodes:
        if G.nodes[node]['visit'] == False:
            component = bfs(G, node)
            component.sort()
            # print(len(component), component)
            if len(component) > 100:
                components.append(component)
            else:
                smallone = smallone + component
    smallone.sort()
    components.append(smallone)
    
    for i, component in enumerate(components):
        for j, node in enumerate(component):
            G.nodes[node]['subgraph'] = i
            G.nodes[node]['at'] = j

        # building a new graph to contain the subgraph
        subgraph = nx.Graph()
        for node in component:
            for nbr in G.neighbors(node):
                # set weights in new graph, ATTENTION: ONLY if graph is unweighted
                weight = 1
                subgraph.add_edge(G.nodes[node]['at'], G.nodes[nbr]['at'], weight=weight)
        subgraphs.append(subgraph)

    return subgraphs

###generate skipgram training batches

In [0]:
def recursion_find(left, right, target, length):
    if left + 1 >= right:
            # print('left', left)
            return left, target - length[left]

    mid = (left + right) // 2
    if target < length[mid]:
        return recursion_find(left, mid, target, length)
    if target >= length[mid]:
        return recursion_find(mid, right, target, length)

def my_graph_context_batch_iter(paths, FLAGS, batch_size):
    walk_length = FLAGS.walk_length
    window_size = FLAGS.window_size
    if (walk_length < window_size):
        print('ERROR: walk_length %d is less than window_size %d' % (walk_length, window_size))
    pairs_in_path = walk_length * window_size * 2 - window_size * (window_size + 1)
    total_pairs = pairs_in_path * len(paths)
    # print('total_pairs', total_pairs)

    length = np.zeros(walk_length, dtype=int)
    for i in range(walk_length):
        if (i < window_size):
            length[i] = window_size + i
        elif (i + window_size + 1 >= walk_length):
            length[i] = window_size + walk_length - 1 - i
        else:
            length[i] = window_size * 2
    # print('length', length)

    total_length = [0]
    for i in range(1, walk_length):
        total_length.append(total_length[i - 1] + length[i - 1])
    # print('total_length', total_length)

    while True:
        batch = np.zeros(batch_size, dtype=np.int32)
        labels = np.zeros((batch_size, 1), dtype=np.int32)
        for i in range(batch_size):
            idx = np.random.randint(0, total_pairs)
            path = int(idx // pairs_in_path)
            idx = int(idx % pairs_in_path)
            # print('idx', idx)
            a, b = recursion_find(0, walk_length, idx, total_length)
            k = -1
            for j in range(a - window_size, a + window_size + 1):
                if j >= 0 and j < walk_length and j != a:
                    k += 1
                    if k == b:
                        b = j
                        break
            # print('a, b,', a, b)
            # print(type(i), type(path), type(a), type(b))
            batch[i] = paths[path][a]
            labels[i, 0] = paths[path][b]
        yield batch, labels

In [0]:
def generate_graph_context_all_pairs(path, window_size):
    # generating graph context pairs
    all_pairs = []
    for k in range(len(path)):
        for i in range(len(path[k])):
            for j in range(i - window_size, i + window_size + 1):
                if i == j or j < 0 or j >= len(path[k]):
                    continue
                else:
                    all_pairs.append([path[k][i], path[k][j]])

    return np.array(all_pairs, dtype=np.int32)


def graph_context_batch_iter(all_pairs, batch_size):
    while True:
        start_idx = np.random.randint(0, len(all_pairs) - batch_size)
        batch_idx = np.array(range(start_idx, start_idx + batch_size))
        batch_idx = np.random.permutation(batch_idx)
        batch = np.zeros(batch_size, dtype=np.int32)
        labels = np.zeros((batch_size, 1), dtype=np.int32)
        batch[:] = all_pairs[batch_idx, 0]
        labels[:, 0] = all_pairs[batch_idx, 1]
        yield batch, labels

###constructing target neighbors

In [0]:
def construct_traget_neighbors(nx_G, X, FLAGS, mode='WAN'):
    # construct target neighbor feature matrix
    X_target = np.zeros(X.shape)
    nodes = nx_G.nodes()

    if mode == 'OWN':
        # autoencoder for reconstructing itself
        return X
    elif mode == 'EMN':
        # autoencoder for reconstructing Elementwise Median Neighbor
        for node in nodes:
            neighbors = list(nx_G.neighbors(node))
            if len(neighbors) == 0:
                X_target[node] = X[node]
            else:
                temp = np.array(X[node])
                for n in neighbors:
                    if FLAGS.weighted:
                        # weighted sum
                        temp = np.vstack((temp, X[n] * nx_G[node][n]['weight']))
                    else:
                        temp = np.vstack((temp, X[n]))
                temp = np.median(temp, axis=0)
                X_target[node] = temp
        return X_target
    elif mode == 'WAN':
        # autoencoder for reconstructing Weighted Average Neighbor
        for node in nodes:
            # put myself ( maybe can be deleted, because second-order will count itself)
            temp = np.array(X[node])
            # a = temp
            # total = len(nodes)
            # weights = np.array(1)

            # compute neighbors
            # alpha = 0.5
            # # beta = 0
            # first_order = 1
            neighbors = list(nx_G.neighbors(node))
            neighbors_length = len(neighbors)
            if neighbors_length != 0:
                for n in neighbors:
                    if FLAGS.weighted:
                        # weighted sum
                        temp = np.vstack((temp, X[n] * nx_G[node][n]['weight']))
                    else:
                        temp = np.vstack((temp, X[n]))
                        # set_n = set(nx_G.successors(n))
                        # set_n = set_n.intersection(set(neighbors))
                        # second_order = len(set_n) / neighbors_length
                        # # weights = np.append(weights, first_order)
                        # w = alpha * first_order + (1 - alpha) * second_order
                        # nx_G[node][n]['weight'] = w
                        # b = np.array(X[n])
                        # w = np.dot(a, b)
                        # nx_G[node][n]['weight'] = total / (w + 1)
                temp = np.mean(temp, axis=0)
                # temp = np.average(temp, axis=0, weights = weights)
                X_target[node] = temp
        return X_target

##main

###prepare for final train

In [24]:
FLAGS = tf.app.flags.FLAGS
inputEdgeFile = FLAGS.inputEdgeFile
inputFeatureFile = FLAGS.inputFeatureFile
inputLabelFile = FLAGS.inputLabelFile
outputEmbedFile = FLAGS.outputEmbedFile
window_size = FLAGS.window_size
config = Config()

G = read_graph(FLAGS, inputEdgeFile)
print(G.number_of_nodes(), G.number_of_edges())
nodes = G.nodes()
# Total number nodes
N = len(nodes)
dims = FLAGS.dimensions

loading graph...
3312 4660


In [0]:
subgraphs = build_subgraphs(G)
embeddings = []
for s, subgraph in enumerate(subgraphs):
    print ('Subgraph %d with %d nodes and %d edges' % 
           (s, subgraph.number_of_nodes(), subgraph.number_of_edges()))
    if not is_connected(subgraph):
        print("graph picies")
        embedding = train(subgraph, FLAGS, config.max_iters_pretrain)
    else:
        print ('Graph Coarsening...')
        recursive_graphs = graph_coarsening(subgraph)
        embedding_input = None
        for i in range(len(recursive_graphs) - 1, -1, -1):
            # temp = list(recursive_graphs[i].nodes())
            # temp.sort(reverse=True)
            # print(temp)
            # print(len(recursive_graphs[i].nodes()))
            print('recursive_graph level', i)
            embedding = train(recursive_graphs[i], FLAGS, config.max_iters_pretrain, embedding_input)
            if i > 0:
                embedding_input = prolongate(recursive_graphs[i - 1], embedding, FLAGS.dimensions)
    embeddings.append(embedding)
final_embedding = np.zeros([N, FLAGS.dimensions], dtype = np.float32)
for node in nodes:
    s = G.nodes[node]['subgraph']
    a = G.nodes[node]['at']
    final_embedding[node] = embeddings[s][a]


Subgraph 0 with 2110 nodes and 3720 edges
Graph Coarsening...
len of recursive_graphs and size 2 1407
len of recursive_graphs and size 3 868
len of recursive_graphs and size 4 578
len of recursive_graphs and size 5 365
len of recursive_graphs and size 6 238
len of recursive_graphs and size 7 152
len of recursive_graphs and size 8 96
recursive_graph level 7
training graph with 96 nodes and 342 edges
simulating random walk...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

iterations: 1000, time elapsed: 11.28, loss: 1211.42, 
iterations: 2000, time elapsed: 21.01, loss: 1176.86, 
iterations: 3000, time elapsed: 30.74, loss: 1138.55, 
iterations: 4000, time elapsed: 40.57, loss

### final train

In [0]:

# target neighbors must be constructed before graph context pairs
print('constructing traget neighbors...')
start_time = time.time()
X_target = construct_traget_neighbors(G, X, FLAGS, mode='WAN')  # 对所有结点，重建邻近目标结点
end_time = time.time()
print('time consumed for constructing traget neighbors: %.2f' % (end_time - start_time))

# Perform random walks to generate graph context
node2vec_G = Graph(G, FLAGS.directed, FLAGS.p, FLAGS.q)
node2vec_G.preprocess_transition_probs()
walks = node2vec_G.simulate_walks(FLAGS.num_walks, FLAGS.walk_length)

print('generating graph context pairs...')
start_time = time.time()
all_pairs = generate_graph_context_all_pairs(walks, window_size)  # 游走序列中窗口内的结点对 加入all_pairs
end_time = time.time()
print('time consumed for constructing graph context: %.2f' % (end_time - start_time))

config = Config()
config.struct[0] = FLAGS.feaDims
config.struct[-1] = FLAGS.dimensions

model = Model(config, N, dims, X_target, final_embedding) # attension


In [0]:
print('Final train')
batch_size = config.batch_size
max_iters = config.max_iters_finaltrain
idx = 0
print_every_k_iterations = 1000
start = time.time()
loss = 0

init = tf.global_variables_initializer()
# with tf.Session(config=config_tf) as sess:
sess = tf.Session(config=config_tf)
sess.run(init)
for iter_cnt in range(max_iters):
    idx += 1

    # train for skip-gram model
    batch_index, batch_labels = next(graph_context_batch_iter(all_pairs, batch_size))

    feed_dict = {model.inputs: batch_index, model.labels: batch_labels}
    _, loss_value = sess.run([model.train_opt, model.loss], feed_dict=feed_dict)
    loss += loss_value

    if idx % print_every_k_iterations == 0:
        end = time.time()
        print('iterations: %d' % idx + ', time elapsed: %.2f, ' % (end - start), end='')
        print('loss: %.2f, ' % (loss / idx), end='')

        y = read_label(inputLabelFile)
        embedding_result = sess.run(model.embedding)
        macro_f1, micro_f1 = multiclass_node_classification_eval(embedding_result, y, 0.7)
        print('[macro_f1 = %.4f, micro_f1 = %.4f]' % (macro_f1, micro_f1))
embedding_result = sess.run(model.embedding)

print('optimization finished...')
y = read_label(inputLabelFile)
print('repeat 10 times for node classification with random split...')
node_classification_F1(embedding_result, y, 0.95)
# print('saving embedding result...')
# write_embedding(embedding_result, outputEmbedFile)

###scoring and output

In [0]:

node_classification_F1(embedding_result, y, 0.7)

###extra train

In [0]:
print('extra train')
for iter_cnt in range(max_iters):
    idx += 1

    # train for skip-gram model
    batch_index, batch_labels = next(graph_context_batch_iter(all_pairs, batch_size))

    feed_dict = {model.inputs: batch_index, model.labels: batch_labels}
    _, loss_value = sess.run([model.train_opt, model.loss], feed_dict=feed_dict)
    loss += loss_value

    if idx % print_every_k_iterations == 0:
        end = time.time()
        print('iterations: %d' % idx + ', time elapsed: %.2f, ' % (end - start), end='')
        print('loss: %.2f, ' % (loss / idx), end='')

        y = read_label(inputLabelFile)
        embedding_result = sess.run(model.embedding)
        macro_f1, micro_f1 = multiclass_node_classification_eval(embedding_result, y, 0.7)
        print('[macro_f1 = %.4f, micro_f1 = %.4f]' % (macro_f1, micro_f1))
embedding_result = sess.run(model.embedding)
print('optimization finished...')
y = read_label(inputLabelFile)
print('repeat 10 times for node classification with random split...')
node_classification_F1(embedding_result, y, 0.70)

###train only for network structure

In [0]:
print('Train for skipgram model only')
embedding = train(G, FLAGS, config.max_iters_finaltrain, final_embedding)
y = read_label(inputLabelFile)
print('repeat 10 times for node classification with random split...')
node_classification_F1(embedding, y, 0.95)

In [0]:
print('Train for skipgram model only with no harp')
embedding = train(G, FLAGS, config.max_iters_finaltrain)
y = read_label(inputLabelFile)
print('repeat 10 times for node classification with random split...')
node_classification_F1(embedding, y, 0.95)

##the end